In [8]:
# coding: utf-8
import sys, os
# 1. 実行しているファイルから見た「プロジェクトのルート」を計算
# src/feed_forward_network/ 内のスクリプトから見て、2つ上の階層（プロジェクトルート）を指定
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))

mnist_path = os.path.abspath(os.path.join(os.path.dirname(os.path.dirname(current_dir)), 'data', 'feed_forward_network'))

# 2. 検索パスの先頭にプロジェクトルートを追加
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# data/feed_forward_network フォルダと common フォルダを検索パスに追加
from data.feed_forward_network.mnist import load_mnist
from src.feed_forward_network.common.functions import sigmoid, softmax

import pickle
import numpy as np
from data.feed_forward_network.mnist import load_mnist
from common.functions import sigmoid, softmax


def get_data():
    # load_mnist: MNISTデータセットをロードする関数
    ## normalize=True: 画像データを0-1の範囲に正規化
    ## flatten=True: 画像データを1次元のベクトルに変換
    ## one_hot_label=False: ラベルをone-hotエンコーディングで表現
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test


def init_network():
    # モデルの重みとバイアスを読み込む
    weight_path = os.path.join(mnist_path, 'sample_weight.pkl')
    with open(weight_path, 'rb') as f:
        network = pickle.load(f)
    return network


def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3'] # 重み
    b1, b2, b3 = network['b1'], network['b2'], network['b3'] # バイアス

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

    return y


x, t = get_data()
network = init_network()
accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p= np.argmax(y) # 最も確率の高い要素のインデックスを取得
    if p == t[i]:
        accuracy_cnt += 1

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352


In [9]:

# バッチ処理対応版
x, t = get_data()
network = init_network()

# 0~9999の数字を100個ずつ取り出してバッチ処理
batch_size = 100
accuracy_cnt = 0

# 数学的イメージ
# X (100, 784) * W1 (784, 50) + B1 (50) = (100, 50)
# バッチサイズNが変わっても重みの形は変わらない
for i in range(0, len(x), batch_size):
    # X[0:100], X[100:200], X[200:300], ...
    x_batch = x[i:i+batch_size]

    # 100枚分を一気に予測
    y_batch = predict(network, x_batch)

    # 100枚分の予測結果から最も確率が高い要素のインデックスを取得
    p = np.argmax(y_batch, axis=1)

    # 正解と一致しているかを確認
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352
